In [1]:
from contrans import contrans

In [2]:
import numpy as np
import pandas as pd

In [3]:
ct = contrans()
ct.mypassword

'olivegardenunlimitedbreadsticks'

In [13]:
house = ct.get_votes()

In [14]:
house

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
486659,118,House,1118,31101,6,100.0
486660,118,House,1118,31102,9,99.6
486661,118,House,1118,39301,6,100.0
486662,118,House,1118,39307,6,100.0


In [16]:
house = house[["rollnumber", "icpsr", "cast_code"]]

In [17]:
house_mat = pd.merge(house, house, on="rollnumber")

In [19]:
len(house_mat)

211856536

In [20]:
house_mat["agree"] = house_mat["cast_code_x"] == house_mat["cast_code_y"]

In [22]:
house_mat = house_mat.groupby(["icpsr_x", "icpsr_y"]).agg({"agree": "mean"})

In [23]:
house_mat = house_mat.reset_index()

In [24]:
house_mat

,icpsr_x,icpsr_y,agree
0,14854,14854,1.000000
1,14854,14863,0.830948
2,14854,14873,0.386404
3,14854,15029,0.412343
4,14854,15433,0.366726
...,...,...,...
200607,91980,31102,0.731664
200608,91980,31103,0.190805
200609,91980,39301,0.236136
200610,91980,39307,0.239714


In [25]:
ideo = ct.get_ideology()

In [26]:
members = ideo[["bioname", "icpsr", "party_code"]]

In [27]:
members

,bioname,icpsr,party_code
0,"ROGERS, Mike Dennis",20301,200
1,"SEWELL, Terri",21102,100
2,"PALMER, Gary James",21500,200
3,"CARL, Jerry L.",22108,200
4,"MOORE, Barry",22140,200
...,...,...,...
443,"STEIL, Bryan",21970,200
444,"TIFFANY, Thomas P.",21989,200
445,"FITZGERALD, Scott",22115,200
446,"VAN ORDEN, Derrick",22370,200
